## Homework

## Q1. Install MLflow

To get started with MLflow, install the MLflow Python package.

For this, create a separate Python environment and then install the packages.

Once the mlflow package is installed, run the command `mlflow --version` and check the output.

What's the version?

In [1]:
!mlflow --version

mlflow, version 2.13.0


## Q2. Download and preprocess the data

Use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7

In [2]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet -P data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet -P data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet -P data

--2024-05-28 03:40:32--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 108.138.245.225, 108.138.245.16, 108.138.245.96, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|108.138.245.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2023-01.parquet’

green_tripdata_2023 100%[===================>]   1.36M  1.43MB/s    in 1.0s    

2024-05-28 03:40:34 (1.43 MB/s) - ‘data/green_tripdata_2023-01.parquet’ saved [1427002/1427002]

--2024-05-28 03:40:35--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 108.138.245.225, 108.138.245.16, 108.138.245.96, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|108.138.245.225|:443... conne

In [3]:
!python3 preprocess_data.py --raw_data_path data --dest_path ./output

In [4]:
!ls ./output -1 | wc -l

4


## Q3. Train a model with autolog

The training script, `train.py`, trains a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset. 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

What is the value of the `min_samples_split` parameter:

* 2
* 4
* 8
* 10

In [5]:
!python3 train.py

2024/05/28 03:43:05 INFO mlflow.tracking.fluent: Experiment with name 'autolog' does not exist. Creating a new experiment.
2024/05/28 03:43:06 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'b17f8a2096c94cf38566cd7c98879bd9', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/05/28 03:43:06 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/28 03:43:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/wongcheefah/anaconda3/envs/exp_tracking/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
/home/wongcheefah/anaconda3/envs/exp_tracking/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6.

In [6]:
from mlflow.tracking import MlflowClient

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")
run = client.get_run("b17f8a2096c94cf38566cd7c98879bd9")
run.data.params.get("min_samples_split")

'2'

## Q4. Launch the tracking server locally

To manage the entire lifecycle of our ML model, and get access to a model registry, launch a tracking server.

Task:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

Keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what other command-line argument needs to be passed to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`

## Q5. Tune model hyperparameters

The script, `hpo.py`, reduces the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`.

Modify the script and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (add a few lines of code to the `objective` function) and run the script without passing any parameters.

Open the mlflow UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Don't use autologging. Just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE?

* 4.817
* 5.335
* 5.818
* 6.336

In [7]:
!python3 hpo.py

2024/05/28 03:44:52 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
  0%|                                    | 0/15 [00:00<?, ?trial/s, best loss=?]/home/wongcheefah/anaconda3/envs/exp_tracking/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

  7%|▋          | 1/15 [00:17<04:06, 17.62s/trial, best loss: 5.370086069268862]/home/wongcheefah/anaconda3/envs/exp_tracking/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

 13%|█▍         | 2/15 [00:18<01:43,  7.94s/trial, best loss: 5.370086069268862]/home/wongcheefah/anaconda3/en

In [8]:
from mlflow.tracking import MlflowClient

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

experiment = client.get_experiment_by_name("random-forest-hyperopt")
runs = client.search_runs(experiment_ids=[experiment.experiment_id])

lowest_rmse = float("inf")
best_run_id = None

for run in runs:
    rmse = float(run.data.metrics.get("rmse"))
    if rmse < lowest_rmse:
        lowest_rmse = rmse
        best_run_id = run.info.run_id

print(f"Lowest RMSE: {lowest_rmse} found in run ID: {best_run_id}")

Lowest RMSE: 5.335419588556921 found in run ID: df75c367ec9e4d36adfa97a0e9de29db


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. Promote the best model to the model registry. Use the script, `register_model.py`, which checks the results from the previous step and select the top 5 runs. After that, it calculates the RMSE of those models on the test set (March 2023 data) and saves the results to a new experiment called `random-forest-best-models`.

Update the `register_model.py` script so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

What is the test RMSE of the best model?

* 5.060
* 5.567
* 6.061
* 6.568

In [9]:
!python3 register_model.py

2024/05/28 03:46:54 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2024/05/28 03:46:56 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/28 03:47:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/wongcheefah/anaconda3/envs/exp_tracking/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
/home/wongcheefah/anaconda3/envs/exp_tracking/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/wongcheefah/anaconda3/envs/exp_tracking/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is d